In [1]:
%pip install sconce -q
%pip show sconce


from collections import defaultdict, OrderedDict

import numpy as np
import torch
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchvision.datasets import *
from torchvision.transforms import *
import torch.optim as optim

assert torch.cuda.is_available(), \
"The current runtime does not have CUDA support." \
"Please go to menu bar (Runtime - Change runtime type) and select GPU"

Note: you may need to restart the kernel to use updated packages.
Name: sconce
Version: 0.62
Summary: sconce: torch pipeliner
Home-page: https://github.com/satabios/sconce
Author: Sathyaprakash Narayanan
Author-email: Sathyaprakash Narayanan <snaray17@ucsc.edu>
License: MIT License
Location: /home/sathya/Downloads/sconce
Requires: certifi, charset-normalizer, cmake, contourpy, cycler, fast_pytorch_kmeans, filelock, fonttools, idna, ipdb, Jinja2, kiwisolver, lit, MarkupSafe, matplotlib, mpmath, networkx, numpy, packaging, Pillow, pyparsing, python-dateutil, requests, six, snntorch, sympy, torch, torchprofile, torchprofile, torchvision, tqdm, transformers, typing_extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


Load the Pre-Trained Model Weights

In [6]:
from google.colab import drive
drive.mount('/content/drive')
model_path = "drive/MyDrive/Efficientml/Efficientml.ai/pre-trained_vgg.cifar.pretrained.pth"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:

class VGG(nn.Module):
  ARCH = [64, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

  def __init__(self) -> None:
    super().__init__()

    layers = []
    counts = defaultdict(int)

    def add(name: str, layer: nn.Module) -> None:
      layers.append((f"{name}{counts[name]}", layer))
      counts[name] += 1

    in_channels = 3
    for x in self.ARCH:
      if x != 'M':
        # conv-bn-relu
        add("conv", nn.Conv2d(in_channels, x, 3, padding=1, bias=False))
        add("bn", nn.BatchNorm2d(x))
        add("relu", nn.ReLU(True))
        in_channels = x
      else:
        # maxpool
        add("pool", nn.MaxPool2d(2))

    self.backbone = nn.Sequential(OrderedDict(layers))
    self.classifier = nn.Linear(512, 10)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    # backbone: [N, 3, 32, 32] => [N, 512, 2, 2]
    x = self.backbone(x)

    # avgpool: [N, 512, 2, 2] => [N, 512]
    x = x.mean([2, 3])

    # classifier: [N, 512] => [N, 10]
    x = self.classifier(x)
    return x


#load the pretrained model

model = VGG().cuda()
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

Setup the Dataset

In [10]:
image_size = 32
transforms = {
    "train": transforms.Compose([
        RandomCrop(image_size, padding=4),
        RandomHorizontalFlip(),
        ToTensor(),
    ]),
    "test": ToTensor(),
}
dataset = {}
for split in ["train", "test"]:

    dataset[split] = CIFAR10(
    root="data/cifar10",
    train=(split == "train"),
    download=True,
    transform=transforms[split],
    )

dataloader = {}
for split in ['train', 'test']:
  dataloader[split] = DataLoader(
    dataset[split],
    batch_size=512,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True,
  )




100%|██████████| 170498071/170498071 [00:12<00:00, 13129669.34it/s]


Extracting data/cifar10/cifar-10-python.tar.gz to data/cifar10
Files already downloaded and verified


sconce Configurations

In [14]:
from sconce import sconce


sconces = sconce()
sconces.model= model
sconces.criterion = nn.CrossEntropyLoss() # Loss
sconces.optimizer= optim.Adam(sconces.model.parameters(), lr=1e-4)
sconces.scheduler = optim.lr_scheduler.CosineAnnealingLR(sconces.optimizer, T_max=200)
sconces.dataloader = dataloader
sconces.epochs = 1 #Number of time we iterate over the data
sconces.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sconces.experiment_name = "vgg-gmp"
sconces.prune_mode = "GMP" # Supports Automated Pruning Ratio Detection


Train and Validated the Model on the given dataset

In [16]:
# Train the model
sconces.train()
# Evaludate the model
sconces.evaluate()

train:   0%|          | 0/98 [00:00<?, ?it/s]

test:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch:1 Train Loss: 0.00000 Validation Accuracy: 92.89579


test:   0%|          | 0/20 [00:00<?, ?it/s]

92.89579158316633

Magic Happens here: Compress the model(GMP pruning is set as the prune mode[sconces.prune_mode] above)

In [17]:
# Compress the model
sconces.compress()


Dense_model_size model after sensitivity size=35.20 MiB


test:   0%|          | 0/20 [00:00<?, ?it/s]

Original Model Validation Accuracy: 92.89579158316633 %
Granular-Magnitude Pruning


test:   0%|          | 0/20 [00:00<?, ?it/s]

test:   0%|          | 0/20 [00:00<?, ?it/s]

...

train:   0%|          | 0/98 [00:00<?, ?it/s]

test:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch:5 Train Loss: 0.00000 Validation Accuracy: 93.19639


test:   0%|          | 0/20 [00:00<?, ?it/s]

/n
                Original        Pruned          Reduction Ratio
Latency (ms)    19900.0         19900.0         1.0            
MACs (M)        606             606             1.0            
Param (M)       9.23            9.23            1.0            
Fine-Tuned Sparse model has size=23.18 MiB = 65.85% of Original model size
Fine-Tuned Pruned Model Validation Accuracy: 93.19639278557115


Notice that intially,


*   **Dense Model** has a size of *35.20MiB* and accuracy of *92.89%*.
*   **Post Pruning(GMP) Pruned Model** size *23.18MiB* with accuracy of *65.85%*.
*   Upon **fine-tuning the Prune Model**, we have the final pruned model size of *23.18MiB* with an accuracy of *93.19%*.


| Metric                        | Original | Pruned  | Reduction Ratio |
|-------------------------------|----------|---------|-----------------|
| Latency (ms)                  | 19900.0  | 19900.0 | 1.0             |
| MACs (M)                      | 606      | 606     | 1.0             |
| Param (M)                     | 9.23     | 9.23    | 1.0             |
| Fine-Tuned Sparse Model Size  | -        | 23.18 MiB | 65.85%         |
| Fine-Tuned Pruned Model Validation Accuracy | -    | 93.196% | -   |




